# Inequality Constraint (KKT)

Minime 
$$
f(x_1,x_2) = x_1^2 + x_2^2 -14 x_1 - 6 x_2
$$

Subject to:
$$
g_1(x_1,x_2) =  x_1 + x_2 - 2 \leq 0 \\
g_2(x_1,x_2) =  x_1 + x_2 - 3 \leq 0
$$

Solution can be found by solving the stationary point of the Lagrangien function:

$$
L(x_1,x_2,\lambda_1,\lambda_2) = f(x_1,x_2) + \lambda_1 g_1(x_1,x_2) + \lambda_2 g_2(x_1,x_2)
$$

The KKT condition:

$$
\nabla L_x = 0 \\
\nabla L_\lambda = 0
$$

where 
$$
g(x) \leq 0 \\
\lambda*g(x) = 0
$$

Let's assume that both constraint are active:

$$
\begin{bmatrix}
2 & 0 & 1 & 1 \\
0 & 2 & 1 & 2 \\
1 & 1 & 0 & 0\\
1 & 2 & 0 & 0 
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
\lambda_1\\
\lambda_2
\end{bmatrix}
=
\begin{bmatrix}
14 \\
6 \\
2\\
3
\end{bmatrix}
$$

The solution vector is $[1, 1, 20, -8]$

The constraint are satisfied and the function value is $-18$.
If we active only the first constraint we get the following system.

$$
\begin{bmatrix}
2 & 0 & 1 & 1 \\
0 & 2 & 1 & 2 \\
1 & 1 & 0 & 0\\
0 & 0 & 0 & 0 
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
\lambda_1\\
\lambda_2
\end{bmatrix}
=
\begin{bmatrix}
14 \\
6 \\
2\\
0
\end{bmatrix}
$$

Which is singular system, but can be solved with a CG method. The solution vector is $[3., -1.,  8.,  0.]$ which respect the constraint and has $f(x^*)=-26$, which is better solution than the previous solution.

If we either deactive all the constraint or deactive only the secong one, the constraints are violated. Therefore, the optimion solution is found when the first contraint is active and the second one is desactive.


In [368]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg
from scipy import optimize



def inner(active_set):
    f = lambda  x1,x2 : (x1-14.)*x1 + (x2-6.)*x2
    g1 = lambda  x1,x2 : x1 + x2 -2
    g2 = lambda  x1,x2 : x1 + x2 -3

    Jf = lambda  x1,x2 : np.array([2.*x1-14,2*x2-6])
    J2f = lambda  x1,x2 : np.array([[2.,0.],[0.,2.]])
    Jg1 = lambda  x1,x2 : np.array([[1.,1.]])
    Jg2 = lambda  x1,x2 : np.array([[1.,2.]])

    x0 = np.array([0.,0])
    H = J2f(*x0)
    DG = np.vstack((active_set[0]*Jg1(*x0),active_set[1]*Jg2(*x0)))
    f0 = Jf(*x0)
    g0 = np.array([active_set[0]*g1(*x0),active_set[1]*g2(*x0)])
    Zeros = np.diag(active_set) - np.eye(len(active_set))
    A = sparse.bmat([[H,DG.T],
                  [DG,None]]).A
    b = -np.concatenate((f0,g0))
    x_aug_sol, info = linalg.cg(A,b)
    return x_aug_sol[:2],x_aug_sol[2:]


active_set = np.array([1,1])

def update_active_set(l):
    return np.array(list(map(lambda li : np.sign(max([li,0])),l)))
    
    
for i in range(10):
    x_sol, l_sol = inner(active_set)
    if min(l_sol)>=0:
        break
    else:
        active_set = update_active_set(l_sol)
    
    

In [369]:

print(x_sol) 
print(l_sol)

[ 3. -1.]
[8. 0.]


In [370]:
np.sign(max([10,0]))

1

In [363]:
kk = lambda li : np.heaviside(max(li,0))

In [364]:
kk(10)

ValueError: invalid number of arguments

In [254]:
x_aug_sol, info = linalg.cg(A,b)
x_aug_sol

array([ 3., -1.,  8.,  0.])

In [255]:
f(*x_aug_sol[:2])

-26.000000000000007

In [256]:
g1(*x_aug_sol[:2])

8.881784197001252e-16

In [257]:
g2(*x_aug_sol[:2])

-0.9999999999999991

Let's try to find a solution for the above problem using an interior point algorithm.
First we define again the Lagrangian function.
$$
L(x_1,x_2,\lambda_1,\lambda_2) = f(x_1,x_2) + \tilde{g_1}(x_1,x_2,\lambda_1,\lambda_2) + \tilde{g_2}(x_1,x_2,\lambda_1,\lambda_2)
$$

Then we define a nonlinear constraint such that
$$
\tilde{g_1}(x_1,x_2,\lambda_1,\lambda_2) = \lambda_1^+ g_1(x) = 0\\
\tilde{g_2}(x_1,x_2,\lambda_1,\lambda_2) = \lambda_2^+ g_2(x) = 0
$$

where $\lambda_i^+$ is a 
$$
\begin{cases}
    0, & \text{if } \lambda_i \leq 0\\
    \lambda_i,       & \text{otherwise}
\end{cases}
$$


The KKT condition:

$$
\nabla f_x + \nabla  \tilde{g} = 0 \\
\tilde{g} = 0
$$

where 
$$
g(x) \leq 0 \\
\lambda*g(x) = 0
$$

In [316]:
f_ = lambda  x1,x2,l1,l2 : (x1-14.)*x1 + (x2-6.)*x2
g1_ = lambda  x1,x2,l1,l2 : np.heaviside(-l1,0)*(x1 + x2 -2) + l1*(x1 + x2 -2)
g2_ = lambda  x1,x2,l1,l2 : np.heaviside(-l2,0)*(x1 + x2 -3) + l2*(x1 + x2 -2)

Jg1_ = lambda  x1,x2,l1,l2 : l1*Jg1(x1,x2)
Jg2_ = lambda  x1,x2,l1,l2 : l2*Jg2(x1,x2)

F = lambda  x1,x2,l1,l2 : np.array(Jf(x1,x2) +  Jg1_(x1,x2,l1,l2) + Jg2_(x1,x2,l1,l2)).flatten()
G = lambda  x1,x2,l1,l2 : np.array([g1_(x1,x2,l1,l2),g2_(x1,x2,l1,l2)])

R = lambda  x : np.concatenate((F(*x), G(*x)))

In [317]:
f_(*x_aug_sol)

-26.000000000000007

In [318]:
g1_(*x_aug_sol)

7.105427357600999e-15

In [319]:
g2_(*x_aug_sol)

0.0

In [320]:
g2(*x_aug_sol[0:2])

-0.9999999999999991

In [321]:
R(x_aug_sol)

array([-2.66453526e-15, -3.55271368e-15,  7.10542736e-15,  0.00000000e+00])

In [322]:
Jg1_(*x_aug_sol)

array([[8., 8.]])

In [323]:
Jg1(*x_aug_sol[0:2])

array([[1., 1.]])

In [324]:
F(*x_aug_sol)

array([-2.66453526e-15, -3.55271368e-15])

In [325]:
Jg1_(*x_aug_sol)

array([[8., 8.]])

In [326]:
G(*x_aug_sol)

array([7.10542736e-15, 0.00000000e+00])

In [327]:
sol = optimize.root(R,np.array([1.,1.,-1.,-1.]),method='krylov',tol=1.0e-12)

In [328]:
sol.x

array([ 7.95171482,  4.40342964, -1.        , -0.90342964])

In [329]:
f_(*sol.x)

-55.124624136524055

In [330]:
g1_(*sol.x)

0.0

In [331]:
g2_(*sol.x)

-5.133671265866724e-13

In [332]:
f(*sol.x[:2])

-55.124624136524055

In [333]:
g1(*sol.x[:2])

10.355144464165587

In [334]:
g2(*sol.x[:2])

9.355144464165587